In [18]:
import numpy as np
from PIL import Image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import os
import pickle

In [54]:
def load_and_preprocess_image(image_path, target_size):
    # Görüntüyü yükleme ve boyutlandırma
    img = image.load_img(image_path, target_size=target_size)
    # Görüntüyü numpy dizisine dönüştürme
    img_array = image.img_to_array(img)
    # Boyutu genişletme
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [16]:
def load_model_and_tokenizer(model_path, tokenizer_path):
    model = load_model(model_path)
    with open(os.path.join(tokenizer_path,"caption.pkl"), "rb") as f:
        tokenizer = pickle.load(f)
    return model, tokenizer

In [51]:
def generate_caption(model, tokenizer, image_path, max_sequence_length):
    start_token = 'startseq'
    end_token = 'endseq'
    max_length = max_sequence_length - 1
    
    in_text = start_token
    for _ in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([image_path, sequence], verbose=1)
        yhat = np.argmax(yhat)
        word = word_for_id(yhat, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == end_token:
            break
    return in_text

In [57]:
image_path = 'C:/Users/mert_/Desktop/xd/realProje/indir.jpg'
model_path = 'C:/Users/mert_/Desktop/xd/realProje/model.h5'
tokenizer_path = 'C:/Users/mert_/Desktop/xd/realProje'

target_size = (224,224,3)  
max_sequence_length = 35   # Caption'ın maksimum uzunluğu

In [59]:
img = load_and_preprocess_image(image_path, target_size)
print(img)
model, tokenizer = load_model_and_tokenizer(model_path, tokenizer_path)
caption = generate_caption(model, tokenizer, img, max_sequence_length)
print("Generated Caption:", caption)

[[[[ 7.  6.  2.]
   [ 7.  6.  2.]
   [ 7.  6.  2.]
   ...
   [51. 30. 11.]
   [50. 32. 12.]
   [51. 32. 15.]]

  [[ 5.  4.  2.]
   [ 5.  4.  2.]
   [ 5.  4.  2.]
   ...
   [56. 34. 13.]
   [51. 32. 15.]
   [48. 30. 16.]]

  [[ 5.  3.  4.]
   [ 4.  2.  3.]
   [ 6.  4.  5.]
   ...
   [61. 36. 16.]
   [51. 29. 15.]
   [40. 26. 15.]]

  ...

  [[ 2.  2.  2.]
   [ 2.  2.  2.]
   [ 2.  2.  2.]
   ...
   [ 2.  2.  2.]
   [ 2.  2.  2.]
   [ 2.  2.  2.]]

  [[ 2.  2.  2.]
   [ 2.  2.  2.]
   [ 2.  2.  2.]
   ...
   [ 1.  1.  1.]
   [ 1.  1.  1.]
   [ 2.  2.  2.]]

  [[ 2.  2.  2.]
   [ 2.  2.  2.]
   [ 2.  2.  2.]
   ...
   [ 3.  3.  3.]
   [ 3.  3.  3.]
   [ 2.  2.  2.]]]]


ValueError: in user code:

    File "C:\Users\mert_\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\mert_\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\mert_\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\mert_\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2381, in predict_step
        return self(x, training=False)
    File "C:\Users\mert_\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\mert_\anaconda3\Lib\site-packages\keras\src\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueErr